In [1]:
import sqlite3  # builtin library
import numpy as np
from kadlu.geospatial.data_sources.fetch_util import storage_cfg
from datetime import datetime

In [2]:
conn = sqlite3.connect(f"{storage_cfg()}kadlu_fetch.db")  # location of database on the disk
c = conn.cursor()  # database connection as python object

In [3]:
# create table
c.execute("DROP TABLE salinity")
c.execute("CREATE TABLE IF NOT EXISTS salinity(val int, lat int, lon int, time int, depth int)")

# build an index on temporospatial params for faster querying
c.execute("CREATE INDEX idx_salinity on salinity (lat, lon, time)")
# note that SQLite has a "unique" index option as well to prevent duplicate entries from multiple queries

write some data to the db

In [4]:
salinity_array = np.array([4, 5], dtype=int)
lats = np.array([44, 50], dtype=int)
lons = np.array([-70, -65], dtype=int)
times = np.array([1000, 1001], dtype=int)
depths = np.array([100, 100], dtype=int)

for ix in range(0, len(salinity_array)):
    c.execute(f"INSERT INTO salinity VALUES ({salinity_array[ix]}, {lats[ix]}, {lons[ix]}, {times[ix]}, {depths[ix]})")

# batch insert is also possible: 

#c.executemany(f"INSERT INTO salinity VALUES (?, ?, ?, ?, ?)", 
#              np.vstack((salinity_array, lats, lons, times, depths)).T)


In [5]:
# update the db with changes made
conn.commit()

In [6]:
# read some data out of the database
query = "SELECT * FROM salinity WHERE lat BETWEEN 40 AND 54 AND lon BETWEEN -72 and -55;"  # gulf st lawrence
c.execute(query)
res = np.array(c.fetchall())
print(res)

[[   4   44  -70 1000  100]
 [   5   50  -65 1001  100]]


In [7]:
# commit the changes made to the database and close connection when done
conn.commit()
c.close()
conn.close()